Import libraries

In [5]:
import pandas as pd
from pathlib import Path
import openai
from tqdm import tqdm
import time
import re
import shutil

In [6]:
try:
    base_dir = Path(__file__).resolve().parent.parent
except NameError:
    base_dir = Path().resolve()

1. Loading & cleaning the data - load a raw CSV file containing product data, remove duplicate entries based on product name and ingredients, and split the 'food_groups_en' column into separate food group and subgroup columns. Then clean and standardise the data, keeping only relevant columns, convert all columns to object type, and ensure the ingredient text is lowercase. Finally, save the cleaned DataFrame to an Excel file for further processing.

In [ ]:
# Input
file_path = base_dir / "data" / "raw" / "raw_products_17052025.csv"
df = pd.read_csv(file_path, encoding="utf-8")

# Remove duplicates
df_clean = df.drop_duplicates(subset=['product_name'])
df_clean = df_clean.drop_duplicates(subset=['ingredients_text'])

# Split 'food_groups_en' into 2 columns
food_groups = df_clean["food_groups_en"].str.split(",", expand=True)
food_groups.columns = [f"food_group_{i+1}" for i in range(food_groups.shape[1])]

# Add the newly created food group columns back to the original DataFrame
df_clean = pd.concat([df_clean, food_groups], axis=1)

# Keep only 'food_group_1' as 'food_group' and 'food_group_2' as 'food_subgroup'
df_clean['food_group'] = df_clean['food_group_1']
df_clean['food_subgroup'] = df_clean['food_group_2']
df_clean = df_clean.dropna(subset=['food_group', 'food_subgroup'])

# Drop all food group columns except 'food_group' and 'food_subgroup'
df_clean = df_clean.drop(columns=['food_group_1', 'food_group_2', 'food_group_3'], errors='ignore')

# Convert all columns to 'object' type
df_clean = df_clean.astype('object')
df_clean['ingredients_text'] = df_clean['ingredients_text'].str.lower()

df_clean = df_clean[[
    "food_group",
    "food_subgroup",
    "nova_group",
    "nutriscore_grade",
    "ecoscore_grade",
    "ingredients_text"
]]

# Output
output_path = base_dir / "data" / "processed" / "1_cleaned_products.xlsx"
df_clean.to_excel(output_path, index=False)

2. Translating and preparing ingredients -  load a cleaned product dataset, sample 500 rows per ecoscore grade, and prepare the data for ingredient translation using the OpenAI GPT-4 Turbo model. Define a batch processing function that sends ingredient lists to the model for cleaning, standardisation, and translation into English, following specific rules. Save clean results incrementally to a new Excel file.

In [ ]:
# Input
file_path = base_dir / "data" / "processed" / "1_cleaned_products.xlsx"
df = pd.read_excel(file_path, engine="openpyxl")

# API key has been redacted for security
openai.api_key = "redacted"

df = df.groupby('ecoscore_grade', group_keys=False).apply(
    lambda x: x.sample(n=min(len(x), 500), random_state=42)
).reset_index(drop=True)

output_path = base_dir / "data" / "processed" / "2_translated_ingredients.xlsx"

# Ensure the column exists
if 'translated_ingredients' not in df.columns:
    df['translated_ingredients'] = None

def clean_ingredients_with_gpt_batch(df, model="gpt-4-turbo", delay=1.5, batch_size=10):
    total_rows = len(df)

    for start in tqdm(range(0, total_rows, batch_size), desc="Processing batches"):
        end = min(start + batch_size, total_rows)
        batch = df.iloc[start:end].copy()
        batch_cleaned = []

        for i, row in batch.iterrows():
            if pd.notnull(row['translated_ingredients']):
                batch_cleaned.append(row['translated_ingredients'])
                continue

            messages = [
                {
                    "role": "system",
                    "content": (
                        "You are a food ingredient preprocessing system. Clean and standardize the provided ingredient list using the following rules. "
                        "Your goal is to output a clean, comma-separated list of English food ingredient names. Follow the steps exactly:\n\n"
                        "1. Remove all numbers and special characters, except for commas and spaces.\n"
                        "2. Remove any extra spaces between words.\n"
                        "3. Translate all ingredient names into English.\n"
                        "4. Normalize synonymous ingredients. For example, use 'rapeseed oil' instead of 'canola oil' or 'rapeseed (canola)'.\n"
                        "5. Expand general terms like 'spices' or 'vegetable oil' into their individual ingredients if they are listed.\n"
                        "6. Break down compound or blend ingredients into their components. For example, if 'chocolate chips' contains 'sugar, cocoa paste, cocoa butter, emulsifier: soy lecithins, aroma', list those individually.\n"
                        "7. Group related ingredients under consistent naming. For example, list 'lecithins' and 'mono- and diglycerides' as 'emulsifiers' but preserve original ingredient words.\n"
                        "8. Remove any disclaimers or allergy warnings, such as 'may contain traces of'.\n"
                        "9. If the ingredient list is unreadable or not in a valid food format, return exactly this text: untranslated or unreadable\n\n"
                        "Output format:\n"
                        "- A single, comma-separated list of clean, lowercase, English ingredient names only.\n"
                        "- No explanations, categories, bullet points, symbols, or additional formatting.\n"
                        "- The output must be plain text, with one line and no newline characters."
                    )
                },
                {
                    "role": "user",
                    "content": f"### Input:\n{row['ingredients_text']}\n\n### Cleaned:"
                }
            ]

            try:
                response = openai.chat.completions.create(
                    model=model,
                    messages=messages,
                    temperature=0.3,
                )
                cleaned = response.choices[0].message.content.strip()
            except Exception as e:
                print(f"Error on row {start + i}: {e}")
                cleaned = None

            batch_cleaned.append(cleaned)
            time.sleep(delay)

        # Assign cleaned results to the correct slice
        df.loc[start:end - 1, 'translated_ingredients'] = batch_cleaned

        # Save after each batch
        df.iloc[:end].to_excel(output_path, index=False)

    return df

# Run the processing
df = clean_ingredients_with_gpt_batch(df)

print(f"\nFinished! All cleaned data saved to: {output_path}")


3. Cleaning ingredient lists - load the translated ingredient dataset, remove rows with null or duplicate translated ingredients, and filter out rows marked as 'untranslated' or 'unreadable'. Define a function to recursively extract root ingredients from nested ingredient lists, apply this function to clean and flatten the ingredient lists, and save the resulting DataFrame with cleaned root ingredients to a new Excel file.

In [ ]:
# Input
file_path = base_dir / "data" / "processed" / "2_translated_ingredients.xlsx"
df = pd.read_excel(file_path, engine="openpyxl")

# Root ingredient extractor function
def extract_root_ingredients(text):
    if not isinstance(text, str):
        return ""

    def extract_recursive(s):
        if '(' not in s:
            return [x.strip() for x in re.sub(r'\b\w+:\s*', '', s).split(',') if x.strip()]

        result = []
        while '(' in s:
            match = re.search(r'(\b[^,()]+)?\s*\(([^()]+)\)', s)
            if not match:
                break

            # Extract parent and inner content
            inner = match.group(2)   # 'raisins, dried blue grapes...'

            # Recursively extract from inside parentheses
            extracted = extract_recursive(inner)
            result.extend(extracted)

            # Remove the whole 'parent (inner)' from the string
            s = s[:match.start()] + s[match.end():]

        # Now process remaining outer text (without labels)
        outer = [x.strip() for x in re.sub(r'\b\w+:\s*', '', s).split(',') if x.strip()]
        result.extend(outer)
        return result

    ingredients = extract_recursive(text)
    return ', '.join(ingredients)

# Clean and filter
df_clean = df.dropna(subset=['translated_ingredients'])
df_clean = df_clean.drop_duplicates(subset=['translated_ingredients'])
df_clean = df_clean[~df_clean['translated_ingredients'].str.contains('untranslated|unreadable', case=False)].copy()

# Add clean ingredient column (will later be removed)
df_clean['cleaned_ingredients'] = df_clean['translated_ingredients'].str.strip().str.lower()

# Apply root ingredient extraction function & create cleaned root ingredients column
df_clean['cleaned_root_ingredients'] = df_clean['cleaned_ingredients'].apply(extract_root_ingredients)
df_clean['cleaned_root_ingredients'] = 'ingredients: ' + df_clean['cleaned_root_ingredients']
df_clean = df_clean.drop(columns=['cleaned_ingredients'])

# Output
output_path = base_dir / "data" / "processed" / "3_cleaned_ingredients.xlsx"
df_clean.to_excel(output_path, index=False)


5. Copy final dataset to 'final' folder

In [ ]:
destination_path = base_dir / "data" / "final" / "1_cleaned_ingredients.xlsx"
shutil.copy2(output_path, destination_path)

WindowsPath('C:/pg-cert-applied-data-science-public/data/final/1_cleaned_ingredients.xlsx')